# Cats and Dogs

just importing one image of dog to try and import

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from tqdm import tqdm

DATADIR = "C:/Users/Rudra/Documents/Projects/AI studies/Sentdex/Cats and Dogs/PetImages"

CATEGORIES = ["Dog", "Cat"]

for category in CATEGORIES:  # do dogs and cats
    path = os.path.join(DATADIR,category)  # create path to dogs and cats
    for img in os.listdir(path):  # iterate over each image per dogs and cats
        img_array = cv2.imread(os.path.join(path,img) ,cv2.IMREAD_GRAYSCALE)  # convert to array
        plt.imshow(img_array, cmap='gray')  # graph it
        plt.show()  # display!

        break  # we just want one for now so break
    break  #...and one more!

In [ ]:
#print(img_array)

In [ ]:
#print(img_array.shape)

In [ ]:
IMG_SIZE = 70

new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
plt.imshow(new_array, cmap='gray')
plt.show()

In [ ]:
training_data = []

def create_training_data():
    for category in CATEGORIES:  # do dogs and cats

        path = os.path.join(DATADIR,category)  # create path to dogs and cats
        class_num = CATEGORIES.index(category)  # get the classification  (0 or a 1). 0=dog 1=cat
        IMG_SIZE = 70

        for img in tqdm(os.listdir(path)):  # iterate over each image per dogs and cats
            try:
                img_array = cv2.imread(os.path.join(path,img) ,cv2.IMREAD_GRAYSCALE)  # convert to array
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  # resize to normalize data size
                training_data.append([new_array, class_num])  # add this to our training_data
            except Exception as e:  # in the interest in keeping the output clean...
                pass
            #except OSError as e:
            #    print("OSErrroBad img most likely", e, os.path.join(path,img))
            #except Exception as e:
            #    print("general exception", e, os.path.join(path,img))

create_training_data()

print(len(training_data))

In [ ]:
import random

random.shuffle(training_data)

In [ ]:
for sample in training_data[:5]:
    print(sample[0])

In [ ]:
x=[]
y=[]

for features,label in training_data:
    x.append(features)
    y.append(label)

#print(x[0].reshape(-1,IMG_SIZE,IMG_SIZE,1))

x = np.array(x).reshape(-1,IMG_SIZE,IMG_SIZE,1)

In [ ]:
import pickle

pickle_out = open("X.pickle","wb")
pickle.dump(x, pickle_out)
pickle_out.close()

pickle_out = open("y.pickle","wb")
pickle.dump(y, pickle_out)
pickle_out.close()

### using pickle to load data back

In [ ]:
pickle_in = open("X.pickle","rb")
x = pickle.load(pickle_in)

pickle_in = open("y.pickle","rb")
y = pickle.load(pickle_in)

### Now using the pickle data instead and making the model

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
import pickle
import time
import os

gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction = 0.333)
sess = tf.Session(config = tf.ConfigProto(gpu_options = gpu_options))

pickle_in = open("X.pickle","rb")
x = pickle.load(pickle_in)

pickle_in = open("y.pickle","rb")
y = pickle.load(pickle_in)

#to reshape it to 0-1
x = x/255.0

In [ ]:

dense_layers = [0]
layer_sizes = [64]
conv_layers = [3]

for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            
            NAME = "{}-conv-{}-nodes-{}-dense-{}".format(conv_layer, layer_size, dense_layer, int(time.time()))
            
            log_dir = os.path.join("logs",NAME)
            
            tensorboard = TensorBoard(log_dir)
            #run tensorboard: tensorboard --log_dir=logs/
            
            print(NAME)
            
            model = Sequential()

            model.add(Conv2D(layer_size,(3,3), input_shape = x.shape[1:]))
            model.add(Activation('relu'))
            model.add(MaxPooling2D(pool_size=(2,2)))

            for l in range(conv_layer-1):
                model.add(Conv2D(layer_size,(3,3)))
                model.add(Activation('relu'))
                model.add(MaxPooling2D(pool_size=(2,2)))

            model.add(Flatten()) #this converts our 3D features to 1D features
            for l in range(dense_layer):
                model.add(Dense(256))
                model.add(Activation('relu'))
            
            model.add(Dense(1))
            model.add(Activation('sigmoid'))

            model.compile(loss='binary_crossentropy',
                         optimizer='adam',
                         metrics=['accuracy'])
            
            model.fit(x, y, batch_size = 32, epochs=6, validation_split = 0.3, callbacks=[tensorboard])

In [ ]:
model.save("64x3-CNN.model")

# USING the saved modle to now make predictions

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

testpre = x[0].reshape(70,70)
plt.imshow(testpre,cmap='gray')

#print(x[0])

def prepare():
    tp = x[0].reshape(-1,70,70,1)
    return tp
data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAPsAAAD7CAYAAACscuKmAAAABHNCSVQICAgIfAhkiAAAAAlwSFlzAAALEgAACxIB0t1+/AAAADh0RVh0U29mdHdhcmUAbWF0cGxvdGxpYiB2ZXJzaW9uMy4xLjAsIGh0dHA6Ly9tYXRwbG90bGliLm9yZy+17YcXAAAgAElEQVR4nO2de9BeVX3vv4sEFPFCwjWSQCCGACoExAtFkQZQrNaedvRUT8d6LIozotKxR4uHGWtnTlsdtVXHjpfRVlo9WASLlLaIRVBPRSBIuIZIxAAhaESleOWWdf54n2fls1f2d7/7TfI+L7B/35lMfs9+997rsvd6nu/vt36XlHNWIBB4/GOXue5AIBCYDGKxBwIDQSz2QGAgiMUeCAwEsdgDgYEgFnsgMBDs0GJPKZ2aUlqXUlqfUjprZ3UqEAjsfKTt3WdPKc2T9F1Jp0jaKOkaSa/NOd+y87oXCAR2FubvwLXPk7Q+53y7JKWUviDpdyTZxb7HHnvkBQsW9G5gy5Ytjc8ppVa5z/XufB6v2yMeeeSRIv/6178u8i67bCVH8+bNaz3edV9+2bovXvbRnTN//tZHyb52YbfddmttY6bz3BduHH2Ob08/uuZ9urYJPsvt6Zd7x/r0r8Z08/OTn/xEP//5z1s7tSOL/QBJd+HzRknP77pgwYIFOuOMMyQ1J9AtjJ///OeN63ffffci93k5f/nLXxZ51113bW2P5z/wwAO27/fff3+Rb7311iI/8YlPLPKee+5Z5Cc84Qm97ssvjoceeqi1j/wS4cPmot5rr72K/Itf/MK29/DDDxd56dKlReb8OJl9Ytucw3phELzXgw8+2Hqc/eMXGM/hO1K3x36xDffF774kOc9PetKTGm2wTX5huufEvv/qV78qMp9T148O2+O71Nb2Bz/4QTnsiM7e9u2xzVdjSun0lNLqlNLqrpcwEAjMLnbkl32jpCX4vFjSpvqknPOnJH1KkpYsWZLH30b8FvzZz35WZP468hxpiqKM8bSnPa3I/OYkG+CvK3+B+e3IX/+a/vKb032b81ufX2b8xd5jjz2K/NOf/rTRBq+nvPfeexf57rvvLjJ/+Ti+//qv/yoyf9Ge+tSnNtrjvBG8l6P3jnlwDnmf+l5O/WF/HXPpo75InonwF5X3dSyG53DcdV84dr5jBJ8/5+fJT35ya/8oS9sy3DH4LozZQJcNbkd+2a+RtDyldHBKaTdJr5F00Q7cLxAIzCK2+5c95/xwSumtkr4iaZ6kv8s537zTehYIBHYqdoTGK+f8b5L+re/5u+yyS6GqpEZPecpTikyKTdoneVpN6kJax+tJhRwlrykQ6aW7nnSPFI3X8hy2V/eRagvpvjNg8ThVAKciSdKPf/zjIpNGkq6zT65tR01r1ctRYz4nqiAEx0QDKXd06mdGVYq0mn3kNXzfeJzPmOOu7+XOq69pa49zSLlWTaiKcR54r3GfunYHwoMuEBgIYrEHAgPBDtH4mWLLli2FepAykfrRGl9TIX4mfXJUmJSZe5e8j9snlTwVJ3icbZCCUgWoKRrngdeTjvI4+0hViDLbqPeIFy5cWGRSxz7OIV3U1t2Hc+hUJu48sO9u3tycS835cZZyt8/u5rDLqaZWNae7L9UaqliU6U8iNcfI3RQ+v/G463eYiF/2QGAgiMUeCAwEE6fxY2siLZqkPKSHtaXVudWS9tCyz3s5N1reh8elJgXeuHFjkUnLDjzwwGnvRat33QbHxPuybV7j3FHZNq20taPHfffdV2TSQF7vXH1JETmfpKDLli1rtEe1gX9bu3ZtkTnWFStWtPbjggsuaO1HTVudNd7FIFAl4L26dlCcKzPBZ+kcXZwLcO3Ew8+cd7Y9HutsOdUEAoHHEGKxBwIDQSz2QGAg2O7kFduDBQsW5JNOOklSU5+ifkQ9cvHixY3r6yCLMagTu60pwm2F1PowdWUGo3B7cP/992+9nvelh1i9ZUWdmOdt2rQ1pojj41aPC4rh+Gp9k32nJ1qXJ+EY1K15Du9Ze+xxHHzmbuuU83nooYcWmbYR3nPNmjWN9jgOt/3lthnd9mo9H27bkc/cxZ3zHGcPqdtz27Nt13zoQx/SXXfd1TrA+GUPBAaCWOyBwEAw0a03aSsNIbUhvSNNWb16deNalxGFFI8BJAwgoHpAKrXPPvsUuab9blumLY5Y6keF63hyXr/ffvsVmRSW/WDb3GbavHlzkbm9Vm/1kXJzvIyhJv3ltibp9jOe8Ywi/+hHPypynZXn9ttvL7KLW1+/fn2RX/SiFxX561//epGXL19e5FWrVhV55cqVjfZuuumm1jHx3eEculRibktUaqpM7r5u29e9F5zbWl3lvPFvbanSulJdxS97IDAQxGIPBAaCidL4nHOhJC5O2tHi+jwXxEFrsPM8c95NtbeZSwZJqsqkjS4gh3SrboN02MU3816k2FRTSM9doIck3Xjjja1j4lzzOOOnn/nMZxb5m9/8ZpEZI19TUD7Pu+7amp/UUV4m86Qqc8MNNxT58MMPb+231LTmsz1n7SZF5jviLPl1f+mx6YJcnMcf7+NU2bq/lKlijduOQJhAIBCLPRAYCibqVLNw4cJ88sknS/L5r7usoI4mH3zwwUX+wQ9+UGRnmXTx7LVDCOeGbZO6uZRMfdQUyTt7HHDAAUV2c0Uad8ghh7SeUzsK0YLP8bEf7CMt2ldffXWRmemXuwiUpaYKw10C0mc6LLE9p76wfyeccEKjPVrtOVaXt8Dlk3fnSP3yIfCZuVh8yk6FkDyNb1NHPvKRj2jjxo3hVBMIDBmx2AOBgWCi1viUUqE9tCS7lFE1fSJlprXaxXi7skuuwEHtM820SPQL5/XOX9yVR+qqX+foHvtIVYPX0pmoKw3WMccc03q9o6OcZ1rEaZl/+tOf3nqt1FRnGOvA8XFHgqqJK9Rw881bM5Zzd0GSfvjDHxb56KOPLrKbW/dsOI56h8EVn+B5M63j5nYI6vaco9fYmt9VfmvaX/aU0t+llDanlG7CsYUppa+mlG4b/d+/WmMgEJgT9KHxn5V0anXsLEmX5ZyXS7ps9DkQCDyKMS2Nzzl/I6W0tDr8O5JOHMnnSLpC0p/2aXBMb+is4eh5bY2n4wHp87333tt6vaPrLmSwhrPIuvRM7hyXYVXyRS36FLLgmJw1vR7fJZdcUmT66TN9FCvCuvRffBZsu6bx7DvnwWUBJu132YGf85znFJmUXpLuueeeItNS7zLCOucZp1rW11AddRmB+9SWc32Sms+Zf2sb02z4xu+Xc75Hkkb/77ud9wkEAhPCrBvoUkqnSzpd2jaHeSAQmBy2d7H/MKW0KOd8T0ppkaTN7kSWbF6wYEEeUzmXLYTZWuoaYM4KSrrHa2o6NAYzvzinj7o9UljSUVJeV+CA960tu660s6N7zp+9zTIrbTvPjs6yj7RoU93iXHEOiVpNodpR/62t7+wfZZeJh3Mm+XgIlxXGZS1yDkeSz+TrstOQ3rt56ypK4bI6Ub0YXz8btd4ukvT6kfx6SV/ezvsEAoEJoc/W27mSrpS0IqW0MaV0mqT3STolpXSbpFNGnwOBwKMYfazxrzV/OmmmjaWUCkUhFSL9IS2raTx1fmcNJt2jA4qzlJPK1jTT1Q0jFWPxCB7nvfbdd6v9snb0YRscH33PmU2HdJu0k+gKE3ahnq4uGik9jzsqzHDXGqSqbteFFNZRXs5TPZ9sn5TWhUS7pKSkyFQZ67+5ZJIu643LCNT1zFyGprYsO1GyORAIxGIPBIaCiSecHNMMR1toxa6zupA+ucwxBGkSzyG9Jy2u23N+xs4Kzj6xDWaRoXW7hgvhZQios9q6nPXOAl7/zfn1U51gn6644ooiO2eSrvu+5jWvKfLll19eZDrFuKSk7Hf9zJYsWTJtv9yuh5vPug0XFu5CotmGSz7JvtZqA9UWqqlt70vUegsEArHYA4GhIBZ7IDAQTFRn33XXXUvaIuob1FGc7tkX1M3o6ea2Obq8zXgN79WnhpyzF3QVHKBO69rjdhSPMyCDto06Ntp5FRKck2c961lFZjquSy+9tMjO01Bq2ijYNsfttsv4jlBmG4yrl5o6/7p164pM2wNBHZdbpIyx57hruEAf9pFtuOAX5yVXX89n6zz+HOKXPRAYCGKxBwIDwURp/AMPPKA777xTUpOCkMJwm8pRL6lJk5ynFWkStzbcVhjPkZrbgC7Y5qCDDioyU0NRVWB7jL2XmtSWFM2lwSLFIxUmhXTpnCRfU8zRZ17vyj0z3RSps9ScN26Lsb8s+nDddde1Xusy9zKrrtRUCVmDzoFzy+fHOoO16uNUL2bW5fYe8wbwWpeZuMvr0WWtrWv6tSF+2QOBgSAWeyAwEEzcg25Me0h/aO0kra6t8aTZpLmk246m8nxSHp5Paio1veucBxbpL9vgOaRly5Yta7ThsumyX4Sj6C7+vU4TxX7Ry4995/WkjaS8v/d7v9faP9Zak5qqGAtfsB8s8sHj7NOKFSuK7NJVSU21yu3MEK4Om7OyS9uqe2NQvbvjjjuK7Lz/XJqu+j3kvE2X5XiHsssGAoHHB2KxBwIDwcSLRLQ5ddARgpb12lGAlJeWU+eQQBpHxw0XHFJb/12GV97XWdNpFSYVrh13SMWpznDsbofCpXDiHNZWWldLjWNyaZRc1loX0y1JixYtKjJLKFNN+f73v19kqjksh903IzBVOj4/tztCSzmfU1eAlEuL5eaW53Pc7Affz1pt4PrgLlRbIIxTMaT4ZQ8EBoNY7IHAQDBRGp9zLjSbVkUXp15TUFdjzfnQO/rs4t/rYgCkRKRPtDizvzyHx3nf2nfcOXWQPrtdAQf2u1abSEk5bzyPMktgE6SsTF3Vle2VagqP0xrP50oHHabmcmOQmrXsvvWtbxWZdJ301zkjUW2rHZP4bPheuTp1zm/dqUU1jXeFSNoKWUQ8eyAQiMUeCAwFE6Xx8+fPL0UVSKVJ/ehQUGcqJb0hlSOlJGUmRXYJ/LvqopHOkta5Npw6wbbrlENuTKSBpId03HA+83SWqUHKTUs5x8T+uh0CgvNU+8Yz9JbPmQ5PVDtcXTQXVlxj8+at9Uro2HLooYe2tu2calyxCan57rr0U0Tt2NR2X1fHsG7DOTn1QZ+88UtSSpenlNamlG5OKZ05Oh5lmwOBxxD6fDU8LOlPcs6HS3qBpDNSSkcoyjYHAo8p9CkScY+kccXWn6WU1ko6QNtRtnnLli2FHjnnAlp/a8suwetd9haXwdY5qdQ01dE9tsFSx6RVLny0LnzhCku4ks+cE1Jv0l8XdyD5jDYubNNRd46VNJ7llCWVkOau+5Iyk6byObniB3X/NmzYUGSOlTLHyneE6k/tSOPgipXU2Wba+ssxuYy+kt81qXePpsOMSP+oTvvRkq5SlG0OBB5T6L3YU0pPlnSBpD/OOfvk59ted3pKaXVKabXb3w4EArOPXtb4lNKumlron885f2l0uFfZZpZs3muvvfKYnpKOuvpjXaVySZ94Dekl6S8pD51U+AVUUzdSbt6Lll1XItqFvtaW2bayuzXoEMJ7kfq5emK0TktNCusoofO5p+rkrq1VL/q302+d88ldAZeI0iXzrHdQ+AxdUlN3Lz4blzBSas41x8sxsV90luJ9Sc95bV3W28VitO1Q7JBTTZp62p+RtDbn/Nf4U5RtDgQeQ+jzy368pNdJujGltGZ07H9rqkzzeaMSzndKevXsdDEQCOwM9LHG/z9Jrg7sjMs2BwKBucFEPei2bNlSdKc+RftqnYQ6XJ/ifC7m2umY1OUlH9jAfrn7umJ+dZADdXBXUIHtUQdn350+3QWOidtc1PtcXDfHyjj1Wmfnlhe3Aakfu+2r22+/vcgHHnhgkflcap3dbQnyGbi55TNzwVn19Q58zryv25LjOOpgKWfX4bbt2Asx6rMHAoFY7IHAUDBxGt+2107q1VV3nVSTdIg00AUw9KH6XfTM0S9SLEf9qJrUNIufeR7H4WLNOQ6nTtTBEq4+eB/vMVJ0Bq+wvY0bNzauOeqoo4rsCj3w+vXr17f2iW1zTNyWlJpbbMuXL2+9hnBz2wW+l5RdCi/OM8936dHqzMLsuwty6lMXMX7ZA4GBIBZ7IDAQTJTGz5s3r1hnaYWm5XLvvfcucl2ri7HuDBbgvVxwgQtAcFlqJZ9yinBBOI5K18ELztrtUh/V3lVtcCqE5LOhsm0WgGBmUxYr+NKXvlRk1mr79re/3WiP88bnxzlh8Mrdd99d5OOOO67IrvDFH/7hHzbaYwz75z//+SJzfJzDPoU2anBOSatdOWZXWtmlVqOnYd2eU8OiZHMgECiIxR4IDAQTr/U2tlLSmuus8bV1lNTIOZTQik1q4yzBdPqoHV54nsvqSorFcdCBhONgYIjUtM6yPc4P+86x1rHqY5Cm1tbqww47rMik62yPY12zZk2RP/nJTxaZGXapDhx77LGN9m699dYisxiEywmwcuXK1nNIw12tPalZ6+1jH/tYkT/72c+2jsntxnAOa9WL17hCG0QfB6KuDMSEe9/G8xC13gKBQCz2QGAomDPfeFrdSWdcTLDkrcfO79nFaLsiCrUPNCmzi5Pmcefb7tJN1SB15DXs7+LFi4t82mmnFZmWbhfHL/lYd57H9i688MIi33bbbUV2ddhqpw86tnDeXDZbZx2nysL7vP/972+0x3l4y1veUmTOFef5K1/5SpG/+MUvFpnzXzt3ufwL7r3g9a6QiMsbIPmdgTaf+SgSEQgEYrEHAkPBxGu9jakLHWacVbmmM84/mnBhra74QFcaLOcP7+rU0bJPazUz5na1QbpHuv7e9763yPfee2+R6ZBD2aXpkprqBSk32+aOBn3VCbY3LvwhddfL6+O/zbmlqkfq/PWvf73ItSPUJz7xiSLz/eH7xn68+MUvLvKLXvSiIv/Hf/xHkelAVLfJ+eS887nyPeTzd7ssXfX8nKrg3lUiftkDgYEgFnsgMBBMlMbvsssuxefbJbgn7awt187SSGrk/NBdBg9nNa/vS2pLv/UTTjih9XxSetKyL3+5mZeTtPXtb397kWkxvuGGG4pMlYdjIq2mxZ0OK1KTrrO/9Mdmxlzei+P+5je/WeRVq1a1jkfy/uKugIcLr2X5ZaJ2UmKI7YoVK4rMsXJuOVa+b6ecckqRX/nKV9q+/OM//mPr9ZRdAQ+Om3Nbq6+8l3MCG7/3kakmEAjEYg8EhoKJh7iOLeouzNA5yEg+kwuvIY3n+c6CTjrZVRaYdJH1zNieszyT6tdlqF/2spcVmeGkl1xySZFJ0UlNly1b1toPOm6wRLMk7bfffkXm2Hk9++gSg9JSfsUVVxT5pS99aaM97qA4KzaPM3kl/ep5PsdXzydpPJ8ZnwdVGdJqvheu+IckHXHEEUX+6Ec/WuRzzjmnyFdffbXa4DIK8X2udy3c+9qWDafLkt+nSMQTU0pXp5SuH5Vs/vPR8YNTSleNSjb/U0qpPcdOIBB4VKAPjX9A0qqc81GSVko6NaX0Aknvl/Q3o5LNP5V0Wsc9AoHAHKNPkYgsaWyq3XX0L0taJel/jI6fI+m9kj7edS+WbKajCS3GdblawlnqaaEkFXfZQng+aWPtmMA2Xvva1xb5+uuvb72G97rllluKzESZb3rTmxptkNadffbZrW2QdjKEk/fiHJI6019falrXncr0ne98p/U4wbmhA9E3vvGNxnl8nvRbJ211VN85W/EZ19b/o48+ushux8c5vzg1rraOE1SxXvWqV7X24+Mfb18WbJvj4DOSmiov10CbQ9gO+8anlOaNSj9tlvRVSd+TdF/OeTybGzVVsz0QCDxK0Wux55wfyTmvlLRY0vMkHd52Wtu1UbI5EHh0YEZbbznn+yRdIekFkvZMKY053mJJm8w1n8o5H5tzPrZPssRAIDA7mFZnTyntI+mhnPN9KaXdJZ2sKePc5ZJeJekL6lmymfHsLgUTt8u6kva7Lw6X6oe6TJ21doy6ThljsT/96U8XmXooPc/YNrdAuEVWb00xsOXII48sMnVBZsClbs4AlDZvKsnrvZKfX25nuTpjLgCEernka/LxPNoOZprmqbazOG81l13YbT9y3HW6MgYQ1Zlgx+AW52/8xm8U+cYbb2xtm+3VdgjC2arGdoWufAl99tkXSTonpTRPU0zgvJzzxSmlWyR9IaX0fyRdp6ka7oFA4FGKPtb4GyQd3XL8dk3p74FA4DGAOSsSQbgAiXorhJ5TrqabS6jPNlyGzjqlEqnmTTfdVGRujVCdILV1sdv11hS3rRhgQc83UnGmV6L6w35w3LVqwjG6WmOkpq5MM8dKw2sdP+/UMrctRjh677zQ6r647MB9ynrzPvVWGMdBldBlMH75y19eZG5rugIh9Xvvtpz5/McqS1dce/jGBwIDQSz2QGAgmHh22TGddvSwyyrpaluRMrm0S6S5rsRzTXlJ3UmPGKDBOGQGqdBrjl5vta8BrbNMi7R27doiM0CGlO72228v8jOe8Ywiu9RFkg+koAXeBSm5VGCk6jWNdOqMsxq74+yr8yiTmgEsfEemK65Qn0/qXu9akH5zftatW1fkRYsWtfadOQuYGZfvHscgNefEqazjdRPx7IFAIBZ7IDAUTJTGz58/X/vuu6+kJl0n5a0dGAgX8+uKMDDmmnSW9J6OKUcddVSjPd7rX//1X4vMXQFSd4IW7U2btjoX1rXX2HeOifPDXQGmteJcfe973yvyC1/4wiIffPDBtj1SQhaDcM4opLNsu8v5iXBWcCc7uGytUpN+cxeD6pMLFnHqZG2Np8rDd4QZaX//93+/yNxZ4fl8Ni5gSGo+j64CINMhftkDgYEgFnsgMBBMlMY/8sgjxZJJSu4cJmpLq6OXLiaZ1I00nlSR/XjnO9/ZaI/lg6kSkGax/hmttIccckiRuatQW0vZX8bA8xr6zzNm/gMf+ECRWdeMalHtxMS+33nnnUV2Pt4ztZrX4+tDy6kGtKVaqu/bVbaa88Nn5iIuXW1BvlOMTZCapadJxekPf8011xT50EMPLTKfx+te97oi/+Vf/mWR6SwjeYcwvtPjuYpab4FAIBZ7IDAUTLzW25hOOYuq82ceXz8GaS5TL5HmuPBKUlZaNBmKKEk333xzaxukYvSNJvU77LDDWq8ltayvZ78osx+XXnppkQ84YGtyoDe84Q1FpvWX6b+k5vxcdtllRXYOM65enqOLNY3nNWzDtUe8+tWvbr0PqTMdjiTpWc96VpFJqzlu7lxwp8RZvek4JTWfJ8teP//5zy8yswPzmVPtYOoxOgrVKhXfN+f7Pj4eND4QCMRiDwSGgonT+DHNcOGAXU4Dzu/dUUJaMV1WTsq1b/yf/dmfFfncc88tMmkWaeszn/nMItMfnhZ/lhuWmrSM92WRAdJZWtqpAvCciy66qMi1kxKvoVWZfSScwwufn7Oa1+B5fLaO3p9//vmtx9lG7YAydtqStlVhxiD15n1psaeTS51dltZ5xjDwmTFWge/tP/zDPxT5d3/3d4t83HHHFfk///M/G+25UGRi3PeuTDXxyx4IDASx2AOBgWCiND6lVGgTremkSV3ZTEhR6J/ukvjTwkkKSmcEUrfTTz+9cT1p4JlnnlnkffbZp8if+czW1Hvsx+c+97kiH3PMMUWufbmpOpCu0eLM0Mm3vvWtRXbFB0gP6XgjNWMB6PPdxzpOdFl9ZxuOektNtcgldCSNZwJPOrNQrsfK+AZa9jmf3Cn52te+VmRmJmJ482//9m8Xmbsk9X1dRpvx+Hao1lsgEHh8IBZ7IDAQTJTG77bbbjrwwAMlNakJnQYcvZeaFmPSUVIuF5pIZwaqA8zjzgw0UjN8laWAaXXnvZhMkPSSVu/aOk6KyD66vPi02rJPpI1UDeo89eedd16RncOM65+DS9oo+V0QZ7Xvio0Yg/SVtLi+nu3x3SF153GX5Yb+9lKTSnOuXZwDnXL4jN3uUp1diPd1cp/n1PuXfVTv7bqU0sWjz1GyORB4DGEmNP5MSWvxOUo2BwKPIfSt4rpY0sslfXr0OWmqZPPY6+EcSf9tNjoYCAR2Dvrq7B+W9C5JY4VjL21HyeaHHnqoBGm4QBhXiEBq6iXU56h3uTh3tw33L//yL0VmbLrUDJJ4xzveUeSrrrqq9V7HH39863Hq+HXgBu9Fr7vFixcXudbhxmD8PMfNzKYMDJGk973vfa33mi04u4BLZdVV5GAMzgcDjqSmTaRPsI7bfqS96Lvf/W7jeurpYxuU1NxKo57OQBi+6wzUoc5+6qmnNtpj8BPh6vs5THtGSukVkjbnnK/l4ZZTo2RzIPAoRp9f9uMlvTKl9FuSnijpqZr6pd8zpTR/9OveWbJZ0qckaeHChXPniREIDBx9Cju+W9K7JSmldKKk/5Vz/oOU0hc1w5LN0lZ662iHy7Dq7iN5uk4mQe80powipasp9nOf+9wiX3nllUW+/PLLW9ujVx8pNrfFli5d2jacbfrIeSDdI71kwARpMdt79rOf3WjjPe95T5Hf/OY3t/aDz6YPrXYBMl3X98ko6wJkSIXrzK/0jmNfKPMavmN8fq7On9TcomPKKT4bV3aZW28cB1OE1TSeWWtdjcM+z2lHnGr+VNI7UkrrNaXDR8nmQOBRjBk51eScr5B0xUiOks2BwGMIc1brjTSVdKQrwIV0j38jlSYtc0ED9Ig69thji1zHDbNoxDnnnFNkWkGpKpAqklbxvn2NlPQKZHZZp+YwTp1t1PTu8MMPLzKpKttwGV4JV/Chi07yXn2yzjrPPI6bBRuk5jzQIs73is+JlJyBTxxfXa+O7x5zB3A++Y5QPeQcMI0Vx8dYemnbUuJjcExj9cDV1pPCNz4QGAxisQcCA8GcxbM7pwpaSmtaTWpLCuPK65JSvvGNbyzyX/zFXxSZaZ4+8YlPNNq74IILinzDDTcUmWoD6RdpIy3iLBhQx7MzFRWvJ1VlGiRac5kSyakQdWqmNWvWFJlx+WybgTu0KnOHgXPg0nzVcH9zATLuvjyHZaslv8vD94WBV6TInGfS8PqZ8R1zqgmPk/bzXlRBeM+68MXb3va2IjPWva1UdlfdvfhlDwQGghQvHQEAABOxSURBVFjsgcBAMPGSzWM/YdJUVxiittg6H3jn9066xuPvete7irxkyZIi0zIrNX2dGTv+9Kc/vciMq6f1n+OjdbV2tmCKI84DqZ9Lg8RzaGUnva/96jmmI444onUc9Pd2aZ7oL06HkBp9yzm3wVF6oq6L1qc9XsM2XGbWeofB7SR1WcLHoHpAmf7zfEZS8zlxHrjrslN84wOBwOMDsdgDgYFgojR+3rx5hZLSh9lZ4Gvq5qygpDC0GNPCTMsnCwmQCtEyLzUttfwb1QlmgaUaQOsvqXSdborhkqR1nIf169cXmXSbtHrZsmVFpoNMDVp6+Qw4VrbN447qMxS4Tru1s2i8qxlXg9ZuXsPnwR0QPht331qdpIrm0oc5FYT9oxrGnYC6HqDLhls730yH+GUPBAaCWOyBwEAwURr/y1/+smRgpdMJKQ8pU11/jJSQ55E6kqaSotMfntSL96ytqaRfdEBhSWRadnkvqhO0ptfWeFI5jsNlxj377LOLzFpopIH00Wa5Z6mZNYft8RnUVHwMV5zhxS9+cZEvvPDCxjUcb5/MMV214sbo41cvNek2ZUer+9QMlKQ77rijyAw/dtSd6gT7QfWH7w7rxNVtMAsRM9iM+xhFIgKBQCz2QGAomCiNl7bSFdLONh9faVtLJ6ktrdikT6SXtLqygAPb4z1Zclfy1nH2i9SfVIwOGjy/dgIhzSWto9PKiSeeWGRa2pmgkufzPnW2n7vvvrvILsOM8xGvyyO3tcHsPpK0YcOG1mtmCuf7P9MadZKvAdfnfKlJ/d0OEfvIvjunGu4W1L74DNvls+FOyXgNdMUmxC97IDAQxGIPBAaCidL4XXbZpdBWZ6Ulnamt47REksKQGtPyTcp7//33F5n+5aRS1113XaM9WmF5PWvC0f/eUX1XLroeB9UOjpUJDy+++OIiM9MJz/nWt75VZM5n/Zn00mWb4XFSVsocK+e/vt7VfXPUsw91r2k8abnzdXcOWa70d51diM5afZJl8hynivIdZhyG1HzHmCyVlH4c6hs0PhAIxGIPBIaCWOyBwEAwUZ199913L7XHXEyx29aSmlsS1A2pU/Eaer2tXr26yNRrGExAvXzc3zGoj1MvdPqtq51dB4ZQT6TMYAgGQnALkQEyjCl39cAlv1Xo4GLYnedYvZXltsZm6jXXF05Xdm279FFErbPT7uK221yfCM4V7TX1fLi68cxhMK4H3+VB12uxp5Q2SPqZpEckPZxzPjaltFDSP0laKmmDpP+ec/6pu0cgEJhbzITG/2bOeWXOeZxo/SxJl43qs182+hwIBB6l2BEa/zuSThzJ52iqUsyfTnfRmOqQCjkaV1NCl76qD3UjpWdsu6Pb9b3oPcbCEswcS0839olbQHUb3B6kpxT7xevpHccS06R0zCDblRNgpjSeKta1117bdvo2qb36bJ+50sOU+9J+9/y76tG1HaeaWdeTc5l1Z/pO8r7ciq63Z6m6veIVrygyy3qPt+G6inT0/WXPki5NKV2bUjp9dGy/nPM9kjT6f9+2C1myuY5iCwQCk0PfX/bjc86bUkr7SvpqSunWaa8YgSWbFy1aFCWbA4E5Qq/FnnPeNPp/c0rpnzVV0PGHKaVFOed7UkqLJG3uvMkIbRTMUb2ua/vQJFIsBo2QYZA61/SO9NJ5wZHu8XyeQ/WjpmjMbsuaYPSsI6XnjgEtuOwHU2jVlmRXxth50xGOYpPS1+NjKisHp070sdJ3BcK4+7r3jTIpdv1eOG9D90664hF8NqTqJ598cqO9k046qbVfVJn2339/STtYJCKltEdK6SljWdJLJN0k6SJN1WWXZlCfPRAIzA36/LLvJ+mfR9+y8yX935zzJSmlaySdl1I6TdKdkl49e90MBAI7imkX+6gO+1Etx38s6aRtr+jGmNKQ8riUQTVFI53tU87XBQXQcs1aYQyQkZq0nPSZlv0VK1YU+dvf/naRTzjhhNZ+1zSXwRAcLyka02DRmYgUksESL3nJS4r8uc99rtFeH2s8HYg4ty4Ta5/AEKlJh/mcnJPL9jjb9Cn00Cfox5X+7rre9depDXR4Yh4AZj+WmnN93nnnFZmBW+M+ufLOUrjLBgKDQSz2QGAgmKhvfM65lVo76tfpIABq485zRQYc/V2+fHnjelq76YhDqzkpHv3seT7pWl2Ign1xZZNdPDRrznE+6KhTOyZRNXFptFwMex/U5ztrtaO8XfHYM0WflFXOKYbqVZ2Oi8+Z43NzxTb4PPgsmf24Tkv17//+70VmfEJbFuCIZw8EArHYA4GhYOLZZcdUyVk0u+D8qftYbV3BAKIOcSTlJhVnhlbeiwUYxiGHkvSc5zynyLWfNR0r6GDD1FeOjjItEaklnYZYjKM+j30nBe1Df50FvYv2sz2nhrldGodaBXQ+9C4Nlnv36KteW7gZ+sx3hj7zVMmWLl1a5EMPPbTIbnejnsOjjtq6GfZXf/VXreeNr+9aC/HLHggMBLHYA4GBYOLW+LGl0dEqV/et/hvhfLZdG44edtFGWqtpwSeVJtVnjbWucry0vJIGcqwuRNbRZ46D2Uil5i4B4Sh2n2ytXZlqnP99X9VtZ6FPiKzbnagzJrHvdNAiRSeN532pAjiHHtJ2qakGUoW84IILijx2+gprfCAQiMUeCAwFc2aNd+WXHZ2UmhZ4l7XGybzWleztY/2VtvWVHoM+5atWrSoyw2i7ar05v2le4+gvnUA4DmazkbwTicNMrdiO6tfX90EfP/n6HemTqYbge8jnyliIug2XpNK9P6TubgeE59Q7KGz/j/7oj4p87rnn2j62IX7ZA4GBIBZ7IDAQxGIPBAaCOQuEcXp6Vzy6i2cn+mSqdbpnXZSQoI3ABT+4LSvGv1Ovl5q6Oa9xGVed5yE98zgH3Parr59pJlaXPZXom1bMoY+e3tfu0OdefB58lvfdd1+R69RefBf4N5ctlnPidPbnPe95Ra51dt6X4+V7xSIhDvHLHggMBLHYA4GBYOI0fkxPHSV0ceeS99Ry23C8nudwK4vxwfSMk5pbMaTbVCcY68ztkzvuuKPIBx98cOsYpGaQBe/VJ16fFJt9qlUFYkfixd21feh9fZ67xm3Jura7xtOnDYLbaE6Nqu/LnAek9HweDExy6iCpex0sxev53p9xxhlFPuus6QsyxS97IDAQxGIPBAaCidL4Rx55pGRNdYEJpMt1fDnpKa2ltVfaGKRbpEKUmWKqLk/F9hjb/uxnP7vIDF5xGW9Z3rimkKTijDWnCuGCUVydOs5NvcPggoaIPkUi3FhruFj1PmWM3X37pIKq4dojrea8uQCeuk2+o27ngs+J7R122GFFppdlTeP7BHH1Qa9f9pTSniml81NKt6aU1qaUjkspLUwpfTWldNvo/wXT3ykQCMwV+tL4j0i6JOd8mKZyyK9VlGwOBB5TmJYDpZSeKukESf9TknLOD0p6MKU045LNDz74YMmi6VIGOYcFqZk1lVZQXsOAENIntkcLOOldV10yl6mWVJ8y6R0t/kw9VfeXNJAFKxgb7SztzrmH1FRqjrEPje9Tv6wr+KWPRd29C30cemqVw1m7++wEUOY817shrn2+b3z+7BPPWbhwYZFZrKSOn3e1BV1Zb4c+v+yHSPqRpL9PKV2XUvr0qObbjEs21ylyA4HA5NBnsc+XdIykj+ecj5b0C82AsuecP5VzPjbnfGz9jRUIBCaHPqbMjZI25pyvGn0+X1OLfcYlmx944IFCT7nwSZm6qImz1LqSxo6uOd94qgZ1H0m5aIEnqHawbVrZazgVZtGiRa3Xs48ct6OgdPSpz3M03qkEfXzVu5xc+jj09PGf7/KHd3PiaDzbI0Xms6jniRSdf+O7R9XLjZsqHGVa5qVtd4na2h4/sx3KLptz/oGku1JK4wqGJ0m6RVGyORB4TKHvPvvbJH0+pbSbpNslvUFTXxRRsjkQeIyg12LPOa+RdGzLn2ZUsjmlVOgR6RapNGlnbdBzFJQ0p0/J3r4FDkjX2AbpHukXM7c6p6Frrrmm0UadbXYMWmDr+nBtfSdIQbscQtgvp+b08Xvvssz3ySLrntn2lHXuY3WnusX3jWGpzk9e8o48pN90pHLzyUyx+++/f5Hr581dKK4J9n3x4sWSmjXjaoS7bCAwEMRiDwQGgon6xm/ZsqVQYFI30rUu2sjznHNJH99h56tc0zOXFdRlrd20aVORSS8dbazhnEic9Z/00Kk49Rx2OaRMhz4+811w5/Wx0jt1oKttXkN1i2oYHazce9HlsML3cPPmrRtSLm7BOTzxnPp5U6XjOKharly5UpJ0/fXX277GL3sgMBDEYg8EBoKJ0viUUqHKjoZ3JZXskzSSlMtZm4kuisb7koqvW7euyMxuwz7RwcJZfCVfX4zW4D6OJkTfbDHOSuxqy820H11tE32y8uxoe06tcnESLstRfd+ZFi7hfd0zJlWXmu8M+862x8knu0J+45c9EBgIYrEHAgPBxGu9jemKo4eOstTnOarqnFlcG13Wf9cXWkFZppfW2H333RoE6KyuUpN2udzvPMdZeV2p677Uu49a5NpzoaQ1HMV0Dk/ETLPqSNtmOmqD22WhBXx7ctPzWfKZUeZ74TIe1ffi8+B8jpOVdqml8cseCAwEsdgDgYEgFnsgMBA8KnR2py/WOrvzMHNBEi5O3hWiqHUlts97sR/UuxhrTpkBEgcddFCjDVcf3m3jODsE0WWH6KPDuy3OmerTktf5Z5qiqq/3HrejuC3KLTbq4zzOLdLOLSy0yfYoM3iFxT/4HtH2wyCqGsyl4LzxxmPqehbxyx4IDASx2AOBgWDiNL4NzmuupnquJLJLneTO4XGXgXa6vozBuGXS/nExDKlJ42699dbG9bwvS/CyL9yWceV/HdWv6S/pqSs+QbiCDH3LJhNu69SpB9uToorbbeyv88wjrXZlxGtq7LbCqHoxKIp13Nx2riv9LDVVQr5vVAP6PIP4ZQ8EBoJY7IHAQPCooPHMxElqUgfLOMtwnzRKLo01a8vV1n9Xtpft0eJLrzmWaWY9uTpWecOGDUWmRZbpqvoEqRBdVm8X798nAGV7gmL6ePkRLhbf1TurKS9pMoOR3LvQp7hC3VdX/tmVzeZ7RDXDWeaZokpq7hJwDmmlH7exQ9llA4HA4wOx2AOBgWDOaDypmAu86LqGVKxPgIarIcZ71imjXHlk0jqmAVq2bFnrObwPLe5S07p65ZVXFpkWWGYbJSV01m0XSy15pwwHR6UJF7TTdU0fkPJyHKTuVKMkb0Vnv+hU4wo+cJ5r1cdZ8N2OD2v9HXnkka334bW1auJKflNlGauHQeMDgUAs9kBgKEg7QrNm3FhKP9JUYch7pzt3lrD3HLY91+1H28No+6Cc8z5tf5joYpeklNLqnHNbdZnHddtz3X60Pay22xA0PhAYCGKxBwIDwVws9k/NQZuPhrbnuv1oe1htb4OJ6+yBQGBuEDQ+EBgIJrrYU0qnppTWpZTWp5TOmuW2/i6ltDmldBOOLUwpfTWldNvo/wVd99iBtpeklC5PKa1NKd2cUjpzUu2nlJ6YUro6pXT9qO0/Hx0/OKV01ajtf0optZfk2Tl9mJdSui6ldPEk204pbUgp3ZhSWpNSWj06NqlnvmdK6fyU0q2j537cpNrui4kt9pTSPEl/K+llko6Q9NqU0hGz2ORnJZ1aHTtL0mU55+WSLht9ng08LOlPcs6HS3qBpDNGY51E+w9IWpVzPkrSSkmnppReIOn9kv5m1PZPJZ02C22Pcaaktfg8ybZ/M+e8Eltek3rmH5F0Sc75MElHaWr8k2q7H3LOE/kn6ThJX8Hnd0t69yy3uVTSTfi8TtKikbxI0roJjf3Lkk6ZdPuSniTpO5Kerynnjvltz2Int7lYUy/2KkkXS0oTbHuDpL2rY7M+55KeKun7GtnA5vp9c/8mSeMPkHQXPm8cHZsk9ss53yNJo//3neb8HUZKaamkoyVdNan2RzR6jaTNkr4q6XuS7ss5j6MzZnPuPyzpXZLGESJ7TbDtLOnSlNK1KaXTR8cmMeeHSPqRpL8fqS+fTintMaG2e2OSi70tHOdxvRWQUnqypAsk/XHO+f7pzt9ZyDk/knNeqalf2edJOrzttJ3dbkrpFZI255yv5eFJtD3C8TnnYzSlKp6RUjphltqpMV/SMZI+nnM+WlMu4XNL2VswycW+UdISfF4saZM5d7bww5TSIkka/b95mvO3GymlXTW10D+fc/7SpNuXpJzzfZKu0JTdYM+U0jg+crbm/nhJr0wpbZD0BU1R+Q9PqG3lnDeN/t8s6Z819UU3iTnfKGljzvmq0efzNbX4J/q8p8MkF/s1kpaPLLO7SXqNpIsm2L5G7b1+JL9eU7r0TkeaCir+jKS1Oee/nmT7KaV9Ukp7juTdJZ2sKWPR5ZJeNZtt55zfnXNenHNeqqnn+7Wc8x9Mou2U0h4ppaeMZUkvkXSTJjDnOecfSLorpbRidOgkSbdMou0ZYZIGAkm/Jem7mtIhz57lts6VdI+khzT1zXuapvTHyyTdNvp/4Sy1/UJNUdUbJK0Z/futSbQv6UhJ143avknSe0bHD5F0taT1kr4o6QmzPP8nSrp4Um2P2rh+9O/m8fs1wWe+UtLq0bxfKGnBpNru+y886AKBgSA86AKBgSAWeyAwEMRiDwQGgljsgcBAEIs9EBgIYrEHAgNBLPZAYCCIxR4IDAT/H0B8C1PPwI3YAAAAAElFTkSuQmCC
print(np.argmax(model.predict([prepare()])))